## Noteboook to generate stratified-II splot

#### Dataset: ionic conductivity (For SL models)

**Common log already generated in stratified-split-2 directory**

**Created on February 13th, 2024**

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import os, sys, argparse, pickle, random

In [2]:
%%bash
pwd
ls -ltr

/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/liquid-electrolyte-ML/Science-rev-1/ionic-conductivity_2.1/dataset_v2.0/common-log-output/lgbm_trials/stratified-split-2-cv_2
total 1464
-rw-r--r--  1 riteshk  staff  342499 Feb 13 19:26 train_qsar_stratII_cond.csv
-rw-------@ 1 riteshk  staff  297575 Feb 13 19:26 cond_lgbm_strat2.ipynb
-rw-------@ 1 riteshk  staff  104320 Feb 13 19:28 split_dataset_strat2_cond_sl.ipynb.ipynb


In [3]:
df_train_rand = pd.read_csv('../random-split/train_qsar_rand_cond.csv')
df_val_rand = pd.read_csv('../random-split/val_qsar_rand_cond.csv')
df_test_rand = pd.read_csv('../random-split/test_qsar_rand_cond.csv')
df_train_rand

,solv_comb_sm,salt_smiles,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,...,fr_func_grp_40_salt,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log
0,CCOCCOCCF.O=C1OCCO1,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,1.0000,15.0,0.819544
1,CC1COC(=O)O1,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,0.4800,-0.2,0.468166
2,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,0.4800,-29.5,0.157312
3,COCCOC.COCCOCC(F)(F)C(F)F,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,16,2,279.917842,1.0000,40.0,0.806858
4,CC1COC(=O)O1,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,1.7000,25.0,0.707570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8161,FC(F)C(F)(F)COC(F)(F)C(F)F.COC(=O)OCC(F)(F)F.O...,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,1.2000,25.0,0.477121
8162,CCCCCCCCCC1COC(=O)O1,[Li+].C(F)(F)(F)S(=O)(=O)[O-],0,0,0,0,0,0,0,0,...,0,0,0,0,9,0,148.952573,0.3000,25.0,-1.315155
8163,COCCOCCOCCOCCOC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,16,2,279.917842,1.2700,30.0,0.513458
8164,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,0.6524,59.0,1.092370


In [4]:
df = pd.concat([df_train_rand, df_val_rand, df_test_rand], ignore_index=True)
df

,solv_comb_sm,salt_smiles,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,...,fr_func_grp_40_salt,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log
0,CCOCCOCCF.O=C1OCCO1,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,1.000000,15.00,0.819544
1,CC1COC(=O)O1,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,0.480000,-0.20,0.468166
2,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,0.480000,-29.50,0.157312
3,COCCOC.COCCOCC(F)(F)C(F)F,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,16,2,279.917842,1.000000,40.00,0.806858
4,CC1COC(=O)O1,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,144.964181,1.700000,25.00,0.707570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10191,CC1CCCO1.O=C1OCCO1.CC1COC(=O)O1,[Li+].F[As-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,188.912016,1.000000,-30.00,0.550228
10192,C1COB(OCCOB2OCCCO2)OC1,[Li+].[O-]Cl(=O)(=O)=O,0,0,0,2,0,0,0,0,...,0,0,0,0,6,0,98.949060,1.000000,30.00,-3.790485
10193,CC1COC(=O)O1.CCOC(=O)OCC,CC[N+](CC)(CC)CC.F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,1,16,0,144.964181,1.160764,10.00,0.920819
10194,COCCOCCOCCOCCOC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,16,2,279.917842,2.347062,50.04,0.468790


In [8]:
# df_train_add = pd.read_csv('./train_strat_add.csv')
# df_val_add = pd.read_csv('./val_strat_add.csv')
# df_test_add = pd.read_csv('./test_strat_add.csv')
# df_add = pd.concat([df_train_add, df_val_add, df_test_add], ignore_index=True)
# df_add

,solv_ratio_1,solv_ratio_2,solv_ratio_3,solv_ratio_4,mol_wt_solv_1,mol_wt_solv_2,mol_wt_solv_3,mol_wt_solv_4,mol_wt_salt,conc_salt,temperature
0,0.3,0.7,0.0,0.0,102.031694,88.016044,0.0,0.0,275.123756,0.190698,60.000
1,0.8,0.2,0.0,0.0,102.031694,118.062994,0.0,0.0,217.162493,1.304688,60.000
2,0.8,0.2,0.0,0.0,102.031694,118.062994,0.0,0.0,217.162493,1.154882,10.000
3,0.6,0.4,0.0,0.0,102.031694,118.062994,0.0,0.0,193.984627,0.240000,29.400
4,0.5,0.5,0.0,0.0,90.068080,86.036779,0.0,0.0,151.980186,0.081500,25.000
...,...,...,...,...,...,...,...,...,...,...,...
10191,0.7,0.3,0.0,0.0,102.031694,118.062994,0.0,0.0,151.980186,0.565800,39.300
10192,0.4,0.6,0.0,0.0,102.031694,118.062994,0.0,0.0,217.162493,0.074903,0.000
10193,0.3,0.7,0.0,0.0,88.016044,86.036779,0.0,0.0,194.012536,1.000000,-30.352
10194,0.5,0.5,0.0,0.0,108.068748,88.016044,0.0,0.0,286.933298,1.000000,-10.000


In [5]:
# bins = [-7, -6, -5, -4, -3, -2, -1, 0, 1, 2]
bins = [-7, -4, -3, -2, -1, 0, 1, 2]
labels = np.arange(0, len(bins)-1, 1)
df['conductivity_log_bin'] = pd.cut(df['conductivity_log'], bins=bins, labels=labels)
df['conductivity_log_bin'].value_counts()

conductivity_log_bin
5    6516
4    1631
6    1435
3     496
2      82
1      29
0       7
Name: count, dtype: int64

In [6]:
print(df['conductivity_log'].max())
print(df['conductivity_log'].min())

1.8247114435787173
-6.747146968539926


In [7]:
df_rest, df_test, y_rest, y_test = train_test_split(df, df['conductivity_log'], test_size=0.1, random_state=0, stratify=df['conductivity_log_bin'])
df_test

,solv_comb_sm,salt_smiles,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,...,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log,conductivity_log_bin
1739,CC1COC(=O)O1.O=C1OCCO1,[B-](F)(F)(F)F.CC[N+](CC)(CC)CC,0,0,0,0,0,0,0,0,...,0,0,1,14,0,87.003467,1.139535,60.000,1.455510,6
2150,CC1COC(=O)O1.O=C1OCCO1,[B-](F)(F)(F)F.CC[N+](CC)(CC)CC,0,0,0,0,0,0,0,0,...,0,0,1,14,0,87.003467,0.500000,30.000,1.042752,6
8584,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,8,0,144.964181,0.897900,49.200,0.965672,5
7642,CC1COC(=O)O1.O=C1OCCO1,CC[N+](CC)(CC)CC.F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,1,16,0,144.964181,0.511628,0.000,0.732234,5
9906,C1CCOC1,[Li+].[B-](F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,6,0,87.003467,0.046877,30.000,-0.670613,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,8,0,144.964181,0.565800,39.300,0.956553,5
4441,CC1COC(=O)O1.CCOC(=O)OCC,[B-](F)(F)(F)F.CC[N+](CC)(CC)CC,0,0,0,0,0,0,0,0,...,0,0,1,14,0,87.003467,0.074903,0.000,0.018261,5
7978,O=C1OCCO1.O=C1CCCO1,[Li+].[B-](C(C(F)(F)F)(F)F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,12,1,186.997080,1.000000,-30.352,0.355643,5
4515,C(CCC#N)CC#N.O=C1OCCO1,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,16,2,279.917842,1.000000,-10.000,0.000000,4


In [8]:
req_len = len(df_test)
rat = req_len/len(df_rest)
df_train, df_val, y_train, y_val = train_test_split(df_rest, y_rest, test_size=rat, random_state=0, stratify=df_rest['conductivity_log_bin'])
df_val

,solv_comb_sm,salt_smiles,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,...,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log,conductivity_log_bin
539,C(C(C(F)F)(F)F)OC(C(F)F)(F)F.FC1COC(=O)O1,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,8,0,144.964181,1.000000,25.000000,0.004321,5
4614,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,0,0,0,0,0,0,0,0,...,0,0,0,14,0,186.969171,0.350000,59.000000,0.846329,5
9561,CC1COC(=O)O1.C1COC(=O)O1,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,0,0,0,0,0,0,0,0,...,0,0,0,14,0,186.969171,0.350000,19.500000,0.463270,5
4074,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,8,0,144.964181,0.670000,-10.000000,0.489676,5
1175,C1(C(OC(=O)O1)F)F,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,16,2,279.917842,0.750000,25.000000,0.255273,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6771,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,0,0,0,0,0,0,0,0,...,0,0,0,14,0,186.969171,0.310000,29.400000,0.653011,5
9526,C(CC#N)C#N,C[N+](C)(C)C.C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C...,0,0,0,0,0,0,0,0,...,0,0,1,20,2,279.917842,0.533046,19.989551,0.289963,5
2342,CC1CCCO1.COCCOC.O=C1OCCO1.O=C1OCCO1,[Li+].F[As-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,8,0,188.912016,1.000000,-20.000000,0.789581,5
232,CCOC(=O)OCCF,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,8,0,144.964181,1.000000,19.596000,0.316390,5


In [9]:
df_train

,solv_comb_sm,salt_smiles,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,...,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log,conductivity_log_bin
1134,CC1COC(=O)O1.O=C1OCCO1,CC[N+](CC)(CC)CC.F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,1,16,0,144.964181,0.190698,60.000,0.660961,5
6589,CC1COC(=O)O1.CCOC(=O)OCC,[B-](F)(F)(F)F.CC[N+](CC)(CC)CC,0,0,0,0,0,0,0,0,...,0,0,1,14,0,87.003467,1.304688,60.000,1.379852,6
9285,CC1COC(=O)O1.CCOC(=O)OCC,[B-](F)(F)(F)F.CC[N+](CC)(CC)CC,0,0,0,0,0,0,0,0,...,0,0,1,14,0,87.003467,1.154882,10.000,0.982081,5
932,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,0,0,0,0,0,0,0,0,...,0,0,0,14,0,186.969171,0.240000,29.400,0.598077,5
6318,COCCOC.O=C1CCCO1,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,8,0,144.964181,0.081500,25.000,0.504063,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8453,CC1COC(=O)O1.CCOC(=O)OCC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,8,0,144.964181,1.230000,49.200,0.990060,5
3832,CC1COC(=O)O1,[Li+].F[P-](F)(F)(F)(F)F.COCCOCCOC.COCCOCCOC,0,0,0,0,0,0,0,0,...,0,0,0,26,0,144.964181,0.273190,25.000,0.627673,5
6512,CC1COC(=O)O1,[Li+].[B-](F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,6,0,87.003467,0.000947,25.000,-1.582382,3
6580,CC1COC(=O)O1.CCOC(=O)OC.COC(=O)OC,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,0,0,0,0,0,0,0,0,...,0,0,0,14,0,186.969171,0.800000,-20.000,0.158661,5


In [10]:
df_train['conductivity_log_bin'].value_counts()

conductivity_log_bin
5    5212
4    1305
6    1148
3     397
2      66
1      23
0       5
Name: count, dtype: int64

In [13]:
# zero_columns = df.columns[(df == 0).all()]
# zero_columns

Index(['fr_func_grp_20_salt'], dtype='object')

In [14]:
df_train.to_csv('./train_qsar_stratII_cond.csv', index=False)
df_val.to_csv('./val_qsar_stratII_cond.csv', index=False)
df_test.to_csv('./test_qsar_stratII_cond.csv', index=False)

In [17]:
# random_state = 0
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
# stratify_labels = df_train['conductivity_log_bin']
# curr_dir = os.getcwd()

# fold = 0   
# # Loop over each fold
# rmse_val = []; rmse_train = []
# for train_index, val_index in skf.split(df_train, stratify_labels):
#     print("Fold {}".format(fold))
#     fold_path = curr_dir + '/relu_epoch100_f5_e3_strat_cv_comm_1/' + 'fold_' + str(fold) ## change the directory accordingly
#     os.makedirs(fold_path, exist_ok=True)
#     # Split the data
#     df_train_fold, df_val_fold = df_train.iloc[train_index], df_train.iloc[val_index]
#     df_train_add_fold, df_val_add_fold = df_train_add.iloc[train_index], df_train_add.iloc[val_index]
#     df_train_comb_fold, df_val_comb_fold = df_train.iloc[train_index], df_train.iloc[val_index]

#     file_1 = fold_path + '/train_full.csv'; file_2 = fold_path + '/val_full.csv'
#     # file_1 = 'train_full.csv'; file_2 = 'val_full.csv'
#     file_3 = fold_path + '/train_features.csv'; file_4 = fold_path + '/val_features.csv'
#     # file_3 = 'train_features.csv'; file_4 = 'val_features.csv'
#     # os.chdir(fold_path)
#     print(fold_path)
#     df_train_comb_fold.to_csv(file_1, index=False); df_val_comb_fold.to_csv(file_2, index=False)
#     df_train_add_fold.to_csv(file_3, index=False); df_val_add_fold.to_csv(file_4, index=False)
#     # print(train_index)
#     ## copy val_strat_comv.csv and val_strat_add.csv to the fold directory
#     os.system('cp ./val_strat_comm.csv ' + fold_path + '/test_full.csv')
#     os.system('cp ./val_strat_add.csv ' + fold_path + '/test_features.csv')
#     fold += 1
#     # os.chdir(curr_dir)

Fold 0
/home/ritesh/ExpanDrive/Box/Research-postdoc/liquid-electrolyte-ML/Science-rev-1/ionic-conductivity_2.1/dataset_v2.0/common-log-output/stratified-split-2-cv/relu_epoch100_f5_e3_strat_cv_comm_1/fold_0
Fold 1
/home/ritesh/ExpanDrive/Box/Research-postdoc/liquid-electrolyte-ML/Science-rev-1/ionic-conductivity_2.1/dataset_v2.0/common-log-output/stratified-split-2-cv/relu_epoch100_f5_e3_strat_cv_comm_1/fold_1
Fold 2
/home/ritesh/ExpanDrive/Box/Research-postdoc/liquid-electrolyte-ML/Science-rev-1/ionic-conductivity_2.1/dataset_v2.0/common-log-output/stratified-split-2-cv/relu_epoch100_f5_e3_strat_cv_comm_1/fold_2
Fold 3
/home/ritesh/ExpanDrive/Box/Research-postdoc/liquid-electrolyte-ML/Science-rev-1/ionic-conductivity_2.1/dataset_v2.0/common-log-output/stratified-split-2-cv/relu_epoch100_f5_e3_strat_cv_comm_1/fold_3
Fold 4
/home/ritesh/ExpanDrive/Box/Research-postdoc/liquid-electrolyte-ML/Science-rev-1/ionic-conductivity_2.1/dataset_v2.0/common-log-output/stratified-split-2-cv/relu_epo